# Exploratory Analysis

**CS 5306 - Project 1**

*Team Members: Wenren Zhou (wz366), Alicia Chen (ac2596), Julie Phan (jp2254)*

**Question:** How do user demographics impact contribution and participation on Stack Overflow?

**Data Source:**
Stack Overflow Annual Developer Survey (https://insights.stackoverflow.com/survey)
We will use the annual developer survey from Stack Overflow, which is the largest and most comprehensive survey of software developers and it is made available under the Open Database License.

**Details on Survey (overview of the survey topics):**
There are six sections in this survey, which are: 1. Basic Information; 2. Education, Work, and Career; 3. Technology and Tech Culture; 4. Stack Overflow Usage + Community; 5. Demographic Information; 6. Final Qs. Most questions are optional and the results are anonymized to protect personal information of participants. With nearly 80,000 responses from over 180 countries and territories, this survey collects all-round information of the participants from identity to education background, and their developer experience from programming skills to participation in open source communities.


Some questions that we will explore include: Which demographic groups (age, gender, ethnicity, sexual orientation, disability status, and mental health) have the highest proportion of engagement? The lowest? Which groups are (proportionally) most likely and least likely to be contributors and not just users of the platform? We will also investigate intersectional behavior and trends, such as how non-straight people of color interact with the platform, and how that might differ from heterosexual individuals of color or non-straight and white individuals. Lastly, we will look at how the demographics of Stack Overflow users compare to demographics of other similar spaces (i.e. Quora).

In [1]:
import pandas
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

In [2]:
# loading the dataset
survey = pandas.read_csv("/Users/alicia/FA21/cs5306-p1/survey_results_public.csv")
survey.head()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
2,3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,...,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,...,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN,Appropriate in length,Easy,NaN


In [3]:
# Survey Size
survey.size

4005072

In [4]:
# Parse Data Function - counting for multiple data selections in survey
def simplify_data(demographic_df):
    df_dict = {}
    for index, value in demographic_df.value_counts().items():
        keys = index.split(";")
        for k in keys:
            if k in df_dict:
                df_dict[k] += value
            else:
                df_dict[k] = value
    return df_dict

## Gender

In [12]:
# Gender - raw values of all responses ("check all that apply")
# Survey results do not actually include the "Or, in your own words" responses 
# NaN dropped from dataset
gender = survey['Gender'].value_counts()
gender_df_raw = pandas.DataFrame(gender)
gender_df_raw

,Gender
Man,74817
Woman,4120
Prefer not to say,1442
"Non-binary, genderqueer, or gender non-conforming",690
"Or, in your own words:",413
"Man;Or, in your own words:",268
"Man;Non-binary, genderqueer, or gender non-conforming",252
"Woman;Non-binary, genderqueer, or gender non-conforming",147
Man;Woman,41
"Non-binary, genderqueer, or gender non-conforming;Or, in your own words:",21


In [31]:
# Simplified data for gender 
gender_data = survey['Gender'].copy()
gender_simp = simplify_data(gender_data)
gender_simp

gender_df = pandas.DataFrame(gender_simp.values(), index = gender_simp.keys())
gender_df.rename(columns = {0: 'Count'}, inplace = True)
gender_df

,Count
Man,75428
Woman,4372
Prefer not to say,1442
"Non-binary, genderqueer, or gender non-conforming",1168
"Or, in your own words:",756


## Age

In [7]:
# Age - sorted in the way that it appears in the survey
# NaN values are dropped 
age = survey['Age'].value_counts()
age_df = pandas.DataFrame(age, index = ['Under 18 years old', '18-24 years old', '25-34 years old', 
                                        '35-44 years old','45-54 years old', '55-64 years old', 
                                        '65 years or older', 'Prefer not to say'])
age_df

,Age
Under 18 years old,5376
18-24 years old,20993
25-34 years old,32568
35-44 years old,15183
45-54 years old,5472
55-64 years old,1819
65 years or older,421
Prefer not to say,575


## Ethnicity

In [32]:
# Ethnicity - sorted the way it appears in the survey
'''
Black or of African descent East Asian
Hispanic or Latino/a/x Middle Eastern
White or of European descent
Biracial
Indigenous (such as Native American, Pacific Islander, or Indigenous Australian) South Asian
Multiracial
Southeast Asian
I don't know
Prefer not to say
''' 

ethnicity_data = survey['Ethnicity'].copy()
ethnicity_simp = simplify_data(ethnicity_data)
ethnicity_simp

ethnicity_df = pandas.DataFrame(ethnicity_simp.values(), index = ethnicity_simp.keys())
ethnicity_df.rename(columns = {0: 'Count'}, inplace = True)
ethnicity_df

,Count
White or of European descent,46434
South Asian,9214
Hispanic or Latino/a/x,5570
Southeast Asian,4083
Prefer not to say,3062
Middle Eastern,4222
East Asian,3735
I don't know,2684
Black or of African descent,2686
"Or, in your own words:",2916


In [33]:
survey.head()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
2,3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,...,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,...,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN,Appropriate in length,Easy,NaN


In [94]:
gender_columns = survey[['ResponseId', 'Gender']].copy()
gender_columns['Male'] = gender_columns['Gender'].str.contains("Man")
gender_columns['Female'] = gender_columns['Gender'].str.contains("Woman")
gender_columns['Non-binary, etc.'] = gender_columns['Gender'].str.contains("Non-binary")
gender_columns['Prefer not to say'] = gender_columns['Gender'].str.contains("Prefer not to say")
gender_columns['Or, in own words'] = gender_columns['Gender'].str.contains("Or, in your own words:")

gender_columns = gender_columns.drop(['Gender'], axis = 1)

# operating on the boolean value because it is in theory 1 0 values
gender_columns[['Male', 'Female', 'Non-binary, etc.', 'Prefer not to say', 'Or, in own words']] *= 1

In [96]:
survey_combined = survey.copy()
combined = pandas.merge(survey_combined, gender_columns, on = "ResponseId")
combined.head()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly,Male,Female,"Non-binary, etc.",Prefer not to say,"Or, in own words"
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,...,None of the above,None of the above,Appropriate in length,Easy,62268.0,1,0,0,0,0
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,...,None of the above,None of the above,Appropriate in length,Easy,NaN,1,0,0,0,0
2,3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,...,None of the above,None of the above,Appropriate in length,Easy,NaN,1,0,0,0,0
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,...,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN,1,0,0,0,0
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,...,None of the above,NaN,Appropriate in length,Easy,NaN,1,0,0,0,0


In [101]:
ethnicities = ['Black or of African descent', 'East Asian', 'Hispanic or Latino/a/x', 'Middle Eastern',
               'White or of European descent', 'Biracial', 'Indigenous', 'South Asian', 
               'Multiracial', 'Southeast Asian', "I don't know", 'Prefer not to say (Eth)', 'Or, in your own words']

eth_columns = survey[['ResponseId', 'Ethnicity']].copy()
eth_columns['Black or of African descent'] = eth_columns['Ethnicity'].str.contains("Black")
eth_columns['East Asian'] = eth_columns['Ethnicity'].str.contains("East Asian")
eth_columns['Hispanic or Latino/a/x'] = eth_columns['Ethnicity'].str.contains("Hispanic")
eth_columns['Middle Eastern'] = eth_columns['Ethnicity'].str.contains("Middle Eastern")
eth_columns['White or of European descent'] = eth_columns['Ethnicity'].str.contains("White or of European descent")
eth_columns['Biracial'] = eth_columns['Ethnicity'].str.contains("Biracial")
eth_columns['Indigenous'] = eth_columns['Ethnicity'].str.contains("Indigenous")
eth_columns['South Asian'] = eth_columns['Ethnicity'].str.contains("South Asian")
eth_columns['Multiracial'] = eth_columns['Ethnicity'].str.contains("Multiracial")
eth_columns['Southeast Asian'] = eth_columns['Ethnicity'].str.contains("Southeast Asian")
eth_columns["I don't know"] = eth_columns['Ethnicity'].str.contains("I don't know")
eth_columns['Prefer not to say (Eth)'] = eth_columns['Ethnicity'].str.contains("Prefer not to say")
eth_columns['Or, in your own words'] = eth_columns['Ethnicity'].str.contains("Or, in your own words:")

eth_columns = eth_columns.drop(['Ethnicity'], axis = 1)

# operating on the boolean value because it is in theory 1 0 values
eth_columns[ethnicities] *= 1

In [102]:
combined = pandas.merge(combined, eth_columns, on = 'ResponseId')
combined.head()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Middle Eastern_y,White or of European descent_y,Biracial_y,Indigenous_y,South Asian_y,Multiracial_y,Southeast Asian_y,I don't know_y,Prefer not to say (Eth),"Or, in your own words_y"
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,...,0,1,0,0,0,0,0,0,0,0
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,...,0,1,0,0,0,0,0,0,0,0
2,3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,...,0,0,0,0,0,0,0,0,1,0
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,...,0,1,0,0,0,0,0,0,0,0
